In [33]:
import pandas as pd 
import os 
print(os.getcwd())
data = pd.read_csv("small_data/rob_document_mentions.data", sep="\t")
qrel = pd.read_csv("small_data/rob_entity_ranking.qrel", sep=" ", header=None)

print("data loaded")


/mnt/nfs/work1/smsarwar/sremit
data loaded


In [36]:
qrel.iloc[:,[2]]

,2
0,AIDES
1,AIDS
2,Alfred_A._Knopf
3,Americans
4,Americas
...,...
1245,Vietnam_Veterans_of_America
1246,Vietnam_War
1247,White_House
1248,"Winona,_Mississippi"


In [37]:
print("printing data columns")
column_str = ""
for col in data.columns: 
    column_str+= col + "\t"
print(column_str) 


printing data columns
rob04_qid	query	dbpedia_entity	entity_frq_ttr	rob04_doc	mention	offset	context_size1	context_size2	context_size3	


In [43]:
entity2judgement = {}
print(len(list(set(data["dbpedia_entity"][0:]))))
print(len(list(set(qrel.iloc[:,2]))))
data_entities = sorted(list(set(data["dbpedia_entity"][0:])))
qrel_entities = sorted(list(set(qrel.iloc[:,2])))
print(data_entities[0:10])
print(qrel_entities[0:10])

for index, row in qrel.iterrows():
    qid = row[0]
    eid = row[2]
    judgement = row[3]
    entity2judgement.setdefault(qid, {})
    entity2judgement[qid][eid] = judgement

1721
1038
['14th_Street', '14th_Street_Northwest_and_Southwest_(Washington,_D.C.)', '2_Live_Crew', '@UK', 'AIDES', 'AIDS', 'AIDS_Information_Centre', 'APh_Technological_Consulting', 'A_Charlie_Brown_Christmas', 'Abdullah_II_of_Jordan']
['@UK', 'AIDES', 'AIDS', 'AIDS_Information_Centre', 'A_Charlie_Brown_Christmas', 'Abdullah_II_of_Jordan', 'Acre_(state)', 'Acre_River', 'Adam_Opel', 'Aegean_Islands']


In [50]:
print(len(entity2judgement))

25


In [51]:
#rob04_qid	query	dbpedia_entity	entity_frq_ttr	rob04_doc	mention	offset	context_size1 context_size2 context_size3

import json 

query2doc = {} 
query2entity = {}

for index, row in data.iterrows():
    qid = row['rob04_qid']
    entity = row['dbpedia_entity']
    document = row['rob04_doc']

    query2doc.setdefault(qid, {})
    query2doc[qid].setdefault(document, [])
    query2entity.setdefault(qid, [])
    
    if qid in entity2judgement:
        entity_dict = entity2judgement[qid]
        if entity in entity_dict:
            judgement = entity_dict[entity]
            query2entity[qid].append((entity, judgement))
            query2doc[qid][document].append((entity, judgement))
        else:
            query2entity[qid].append((entity, 0.0))
            query2doc[qid][document].append((entity, 0.0))

In [52]:
for qid in entity2judgement:
    entities = entity2judgement[qid]
    entities = sorted(entities, key=lambda x: x[1], reverse=True)
    entity2judgement[qid] = entities 

In [53]:
for qid in query2doc:
    if qid in entity2judgement:
        documents = query2doc[qid]
        judged_entities = query2entity[qid]

        for document in documents.keys():            
            judged_entities_present = []
            judged_entities_absent = []
            entities = documents[document]
            document_entity_set = set([entity[0] for entity in entities])

            for entity in judged_entities:
                entity_string = entity[0]
                judgement = entity[1]
                if entity_string in document_entity_set:
                    judged_entities_present.append((entity_string, judgement))
                else:
                    judged_entities_absent.append((entity_string, judgement))
            print(document)
            print(judged_entities_present[0:5])
            print(judged_entities_absent[0:5])
            



        

TypeError: 'builtin_function_or_method' object is not subscriptable